In [1]:
from functions import *
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
links = getStaticLinkData()
adj_matrix, upstreams = getSpatialData()

In [3]:
samples1 = getTemporalData("../datasets/gy_link_travel_time_part1.txt")
samples2 = getTemporalData("../datasets/gy_link_travel_time_part2.txt")
samples3 = getTemporalData("../datasets/gy_link_travel_time_part3.txt")
samples = tf.concat([samples1, samples2], 0)
samples = tf.concat([samples, samples3], 0)
del samples1, samples2, samples3

In [4]:
start_date_1 = datetime(2016, 3, 1)
end_date_1 = datetime(2016, 6, 1)
start_date_2 = datetime(2017, 3, 1)
end_date_2 = datetime(2017, 7, 1)

In [5]:
sortedData1 = sortLinks(samples, start_date_1, end_date_1)
sortedData2 = sortLinks(samples, start_date_2, end_date_2)
del samples

## Localised Design

In [ ]:
# temp_data1 = tf.concat([tf.broadcast_to(stat_data, [sortedData1.shape[0], sortedData1.shape[1], stat_data.shape[-1]]), sortedData1], 2)
# temp_data2 = tf.concat([tf.broadcast_to(stat_data, [sortedData2.shape[0], sortedData2.shape[1], stat_data.shape[-1]]), sortedData2], 2)
temp_data1 = sortedData1
temp_data2 = sortedData2
temp_data1.shape, temp_data2.shape

In [ ]:
lr_scheduler = callbacks.LearningRateScheduler(scheduler)
earlystop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, restore_best_weights=True)

###  Pure Temporal Design

In [ ]:
id = 22
train1, test1 = createSeq(temp_data1[:, id, :], INPUT_LENGTH, PREDICT_LENGTH, TEST_LEN)
train2, test2 = createSeq(temp_data2[:, id, :], INPUT_LENGTH, PREDICT_LENGTH, TEST_LEN)

X_train = tf.concat([train1[0], train2[0]], 0)
y_train = tf.concat([train1[1], train2[1]], 0)
X_test = tf.concat([test1[0], test2[0]], 0)
y_test = tf.concat([test1[1], test2[1]], 0)

The rate of missing values in train and test sets

In [ ]:
tf.reduce_sum(tf.cast(tf.equal(y_test, 0), dtype=tf.int32)).numpy() / (y_test.shape[0] * y_test.shape[1]), \
tf.reduce_sum(tf.cast(tf.equal(y_train, 0), dtype=tf.int32)).numpy() / (y_train.shape[0] * y_train.shape[1])

In [ ]:
np.save('../datasets/temporal_x_train.npy', X_train)
np.save('../datasets/temporal_y_train.npy', y_train)
np.save('../datasets/temporal_x_test.npy', X_test)
np.save('../datasets/temporal_y_test.npy', y_test)

In [ ]:
X_train, _, X_test= normalize(X_train, test=X_test)

In [ ]:
X_train = np.load('../datasets/temporal_x_train.npy')
y_train = np.load('../datasets/temporal_y_train.npy')
X_test = np.load('../datasets/temporal_x_test.npy')
y_test = np.load('../datasets/temporal_y_test.npy')

#### LSTM units

In [ ]:
units = [10, 30, 50, 100, 150, 200, 500]
# units = [500, 1000, 2000, 5000, 10000]
# units = [100]

hist_units_mse = []
time_taken_units_mse = []
test_loss_units_mse = []

hist_units_mape = []
time_taken_units_mape = []
test_loss_units_mape = []

for unit in units:
    model = models.Sequential()
    model.add(layers.LSTM(units=unit, return_sequences=False, input_shape=(INPUT_LENGTH, X_train.shape[-1])))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units=PREDICT_LENGTH, activation='elu'))
    model.compile(optimizer='adam', loss=losses.MeanSquaredError())

    start_time = time.time()
    hist_units_mse.append(model.fit(X_train, y_train, epochs=1000, batch_size=360, callbacks=[earlystop, lr_scheduler], validation_split=0.2))
    end_time = time.time()
    time_taken_units_mse.append(end_time-start_time)

    test_loss_units_mse.append(model.evaluate(X_test, y_test))

    model.save(model_path+'simplelstm-unit'+str(unit)+'.keras')

    tf.keras.backend.clear_session()

    # model = models.Sequential()
    # model.add(layers.LSTM(units=unit, return_sequences=False, input_shape=(INPUT_LENGTH, X_train.shape[-1])))
    # model.add(layers.Dropout(0.2))
    # model.add(layers.Dense(units=PREDICT_LENGTH, activation='elu'))
    # model.compile(optimizer='adam', loss=losses.MeanAbsolutePercentageError())

    # start_time = time.time()
    # hist_units_mape.append(model.fit(X_train, y_train, epochs=1000, batch_size=360, callbacks=[earlystop, lr_scheduler], validation_split=0.2))
    # end_time = time.time()
    # time_taken_units_mape.append(end_time-start_time)

    # test_loss_units_mape.append(model.evaluate(X_test, y_test))

    # tf.keras.backend.clear_session()


In [ ]:
time_epoch_mse = []
for i, each in enumerate(hist_units_mse):
    time_epoch_mse.append(time_taken_units_mse[i] / len(each.history['loss']))

plt.plot(units, time_epoch_mse, marker='o')
plt.xlabel('units')
plt.ylabel('Average time taken of each epoch (s)')

In [ ]:
train_loss_units_mse = []
for i, each in enumerate(hist_units_mse):
    train_loss_units_mse.append(each.history['loss'][-1])

plt.plot(test_loss_units_mse, label='Test Loss')
plt.plot(train_loss_units_mse, label='Train Loss')
plt.legend()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

for i in range(len(hist_units_mse)):
    axs[0].plot(hist_units_mse[i].history['loss'], 
                color=colors[i], 
                marker=markers[i], 
                label=f'{units[i]} Units')
    axs[1].plot(hist_units_mse[i].history['val_loss'], 
                color=colors[i], 
                marker=markers[i], 
                label=f'{units[i]} Units')

axs[0].set_ylabel('train loss (MSE)')
axs[1].set_ylabel('validation loss (MSE)')
axs[0].set_xlabel('epochs')
axs[1].set_xlabel('epochs')
axs[0].set_yscale('log')
axs[0].set_xscale('log')
axs[1].set_yscale('log')
axs[1].set_xscale('log')
axs[0].set_ylim(0.5, 7)
axs[1].set_ylim(0.5, 7)
axs[0].legend()
axs[1].legend()

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

for i in range(len(hist_units_mse)):
    ax1.plot(hist_units_mse[i].history['loss'], 
            linestyle=line_styles[0], 
            color=colors[i], 
            marker=markers[i], 
            label=f'{units[i]} Units Train')
    ax2.plot(hist_units_mse[i].history['val_loss'], 
            linestyle=line_styles[1], 
            color=colors[i], 
            marker=markers[i], 
            label=f'{units[i]} Units Validation')

ax1.set_ylabel('train loss (MSE)')
ax2.set_ylabel('validation loss (MSE)')
ax1.set_xlabel('epochs')
ax1.set_yscale('log')
ax2.set_yscale('log')
ax1.set_ylim(6, 15)
#ax2.set_ylim(10.5, 14)
ax1.set_xscale('log')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.show()

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

for i in range(len(hist_units_mape)):
    ax1.plot(hist_units_mape[i].history['loss'], 
            linestyle=line_styles[0], 
            color=colors[i], 
            marker=markers[i], 
            label=f'{units[i]} Units Train')
    # ax2.plot(hist_units_mape[i].history['val_loss'], 
    #         linestyle=line_styles[1], 
    #         color=colors[i], 
    #         marker=markers[i], 
    #         label=f'{units[i]} Units Validation')

ax1.set_ylabel('train loss (MAPE)')
# ax2.set_ylabel('validation loss (MAPE)')
ax1.set_xlabel('epochs')
ax1.set_yscale('log')
# ax2.set_yscale('log')
# ax1.set_ylim(6, 15)
#ax2.set_ylim(10.5, 14)
ax1.set_xscale('log')
ax1.legend(loc='upper right')
# ax2.legend(loc='upper right')
plt.show()

#### Dropout

In [ ]:
def scheduler(epoch, lr):
    if lr < 0.00005: return lr
    if epoch < 5: return lr
    else: return lr * 0.9
lr_scheduler = callbacks.LearningRateScheduler(scheduler)
earlystop = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, restore_best_weights=True)

In [ ]:
dropouts = [0.1, 0.2, 0.5, 0.8]
# dropouts = [0.5]

hist_drop_mse = []
test_loss_drop_mse = []

hist_drop_mape = []
test_loss_drop_mape = []
for dropout in dropouts:
    model = models.Sequential()
    model.add(layers.LSTM(units=150, return_sequences=False, input_shape=(INPUT_LENGTH, X_train.shape[-1])))
    model.add(layers.Dropout(dropout))
    model.add(layers.Dense(units=PREDICT_LENGTH, activation='elu'))
    model.compile(optimizer='adam', loss=losses.MeanSquaredError())

    hist_drop_mse.append(model.fit(X_train, y_train, epochs=100, batch_size=360, callbacks=lr_scheduler, validation_split=0.2))
    # hist_drop_mse[2] = model.fit(X_train, y_train, epochs=100, batch_size=360, callbacks=lr_scheduler, validation_split=0.2)

    test_loss_drop_mse.append(model.evaluate(X_test, y_test))
    # test_loss_drop_mse[2] = model.evaluate(X_test, y_test)

    model.save(model_path+'simplelstm-dropout'+str(dropout)+'.keras')

    tf.keras.backend.clear_session()

    # model = models.Sequential()
    # model.add(layers.LSTM(units=100, return_sequences=False, input_shape=(INPUT_LENGTH, X_train.shape[-1])))
    # model.add(layers.Dropout(dropout))
    # model.add(layers.Dense(units=PREDICT_LENGTH, activation='elu'))
    # model.compile(optimizer='adam', loss=losses.MeanAbsolutePercentageError())

    # hist_drop_mape.append(model.fit(X_train, y_train, epochs=1000, batch_size=360, callbacks=earlystop, validation_split=0.2))
    
    # test_loss_drop_mape.append(model.evaluate(X_test, y_test))

    # tf.keras.backend.clear_session()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

for i in range(len(hist_drop_mse)):
    axs[0].plot(hist_drop_mse[i].history['loss'], 
                color=colors[i], 
                marker=markers[i], 
                label=f'{dropouts[i]} Dropout')
    axs[1].plot(hist_drop_mse[i].history['val_loss'], 
                color=colors[i], 
                marker=markers[i], 
                label=f'{dropouts[i]} Dropout')

axs[0].set_ylabel('train loss (MSE)')
axs[1].set_ylabel('validation loss (MSE)')
axs[0].set_xlabel('epochs')
axs[1].set_xlabel('epochs')
axs[0].set_yscale('log')
axs[0].set_xscale('log')
axs[1].set_yscale('log')
axs[1].set_xscale('log')
axs[0].set_ylim(0.5, 4)
axs[1].set_ylim(0.5, 4)
axs[0].legend()
axs[1].legend()

#### Batch Size

In [ ]:
b_sizes = [10, 30, 120, 360, 720, 1440]
b_sizes = [120]

hist_batch_mse = []
time_taken_batch_mse = []
test_loss_batch_mse = []

hist_batch_mape = []
time_taken_batch_mape = []
test_loss_batch_mape = []

for b_size in b_sizes:
    model = models.Sequential()
    model.add(layers.LSTM(units=150, return_sequences=False, input_shape=(INPUT_LENGTH, X_train.shape[-1])))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(units=PREDICT_LENGTH, activation='elu'))
    model.compile(optimizer='adam', loss=losses.MeanSquaredError())

    start_time = time.time()
    hist_batch_mse.append(model.fit(X_train, y_train, epochs=1000, batch_size=b_size, callbacks=[earlystop, lr_scheduler], validation_split=0.2))
    end_time = time.time()
    time_taken_batch_mse.append(end_time-start_time)

    test_loss_batch_mse.append(model.evaluate(X_test, y_test))

    tf.keras.backend.clear_session()

    # model = models.Sequential()
    # model.add(layers.LSTM(units=100, return_sequences=False, input_shape=(INPUT_LENGTH, FEATURE_COUNT+1)))
    # model.add(layers.Dropout(0.2))
    # model.add(layers.Dense(units=PREDICT_LENGTH, activation='elu'))
    # model.compile(optimizer='adam', loss=losses.MeanAbsolutePercentageError())

    # start_time = time.time()
    # hist_batch_mape.append(model.fit(X_train, y_train, epochs=1000, batch_size=b_size, callbacks=[earlystop, lr_scheduler], validation_split=0.2))
    # end_time = time.time()
    # time_taken_batch_mape.append(end_time-start_time)

    # test_loss_batch_mape.append(model.evaluate(X_test, y_test))

    # tf.keras.backend.clear_session()

In [ ]:
np.save("./trainloss.npy", hist_batch_mse[0].history['loss'])
np.save("./valloss.npy", hist_batch_mse[0].history['val_loss'])

In [ ]:
time_epoch_batch_mse = []
for i, each in enumerate(hist_batch_mse):
    time_epoch_batch_mse.append(time_taken_batch_mse[i] / len(each.history['loss']))

plt.plot(b_sizes, time_taken_batch_mse, marker='o')
plt.xlabel('batch_size')
plt.ylabel('Average time taken of each epoch (s)')

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(10, 10))

for i in range(len(hist_batch_mse)):
    axs[0].plot(hist_batch_mse[i].history['loss'], 
                color=colors[i], 
                marker=markers[i], 
                label=f'B = {b_sizes[i]}')
    axs[1].plot(hist_batch_mse[i].history['val_loss'], 
                color=colors[i], 
                marker=markers[i], 
                label=f'B = {b_sizes[i]}')

axs[0].set_ylabel('train loss (MSE)')
axs[1].set_ylabel('validation loss (MSE)')
axs[0].set_xlabel('epochs')
axs[1].set_xlabel('epochs')
axs[0].set_yscale('log')
axs[0].set_xscale('log')
axs[1].set_yscale('log')
axs[1].set_xscale('log')
axs[0].set_ylim(0.5, 4)
axs[1].set_ylim(0.5, 4)
axs[0].legend()
axs[1].legend()

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

for i in range(len(hist_batch_mse)):
    ax1.plot(hist_batch_mse[i].history['loss'], 
            linestyle=line_styles[0], 
            color=colors[i], 
            marker=markers[i], 
            label=f'B = {b_sizes[i]} Train')
    ax2.plot(hist_batch_mse[i].history['val_loss'], 
            linestyle=line_styles[1], 
            color=colors[i], 
            marker=markers[i], 
            label=f'B = {b_sizes[i]} Validation')

ax1.set_ylabel('train loss (MSE)')
ax2.set_ylabel('validation loss (MSE)')
ax1.set_xlabel('epochs')
ax1.set_yscale('log')
ax2.set_yscale('log')
ax1.set_ylim(6.75, 10)
#ax2.set_ylim(5.25, 6.5)
ax1.set_xscale('log')
ax1.legend(loc='upper left')
ax2.legend(loc='upper right')
plt.show()

### Spatial Features

In [ ]:
upstream_speed1 = np.zeros((sortedData1.shape[0], sortedData1.shape[1], 4))
for i, link in enumerate(upstreams):
    for j, l in enumerate(link):
        if l == -1: continue
        else:
            upstream_speed1[1:, i, j] = sortedData1[:-1, l, -1]
upstream_speed2 = np.zeros((sortedData2.shape[0], sortedData2.shape[1], 4))
for i, link in enumerate(upstreams):
    for j, l in enumerate(link):
        if l == -1: continue
        else:
            upstream_speed2[1:, i, j] = sortedData2[:-1, l, -1]

In [ ]:
temp_data1 = tf.concat([upstream_speed1, temp_data1], 2)
temp_data2 = tf.concat([upstream_speed2, temp_data2], 2)

#### Comparsion

In [ ]:
control_train = np.load("./trainloss.npy")
control_val = np.load("./valloss.npy")

In [ ]:
id = 69
train1, test1 = createSeq(temp_data1[:, id, :], INPUT_LENGTH, PREDICT_LENGTH, TEST_LEN)
train2, test2 = createSeq(temp_data2[:, id, :], INPUT_LENGTH, PREDICT_LENGTH, TEST_LEN)

X_train = tf.concat([train1[0], train2[0]], 0)
y_train = tf.concat([train1[1], train2[1]], 0)
X_test = tf.concat([test1[0], test2[0]], 0)
y_test = tf.concat([test1[1], test2[1]], 0)

In [ ]:
X_train.shape

In [ ]:
np.save('../datasets/local_x_train.npy', X_train)
np.save('../datasets/local_y_train.npy', y_train)
np.save('../datasets/local_x_test.npy', X_test)
np.save('../datasets/local_y_test.npy', y_test)

In [ ]:
X_train, _, X_test = normalize(X_train, test=X_test)

In [ ]:
model = models.Sequential()

model.add(layers.LSTM(units=150, return_sequences=False, input_shape=(INPUT_LENGTH, X_train.shape[-1])))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(units=PREDICT_LENGTH, activation='elu'))
model.compile(optimizer='adam', loss=losses.MeanSquaredError())
model.summary()

In [ ]:
hist_spat_mse = model.fit(X_train, y_train, epochs=200, batch_size=120, callbacks=[earlystop, lr_scheduler], validation_split=0.2)
test_loss_spat_mse = model.evaluate(X_test, y_test)
# tf.keras.backend.clear_session()

In [ ]:
model.save(model_path+'localized.keras')

In [ ]:
model = models.load_model(model_path+'localized.keras')

In [ ]:
model = models.Sequential()
earlystop = callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1, restore_best_weights=True)
model.add(layers.LSTM(units=200, return_sequences=False, input_shape=(INPUT_LENGTH, X_train.shape[-1])))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(units=PREDICT_LENGTH, activation='elu'))
model.compile(optimizer='adam', loss=losses.MeanAbsolutePercentageError())
hist_spat_mape = model.fit(X_train, y_train, epochs=1000, batch_size=120, callbacks=[earlystop, lr_scheduler], validation_split=0.2)
test_loss_spat_mape = model.evaluate(X_test, y_test)
# tf.keras.backend.clear_session()

In [ ]:
plt.plot(hist_spat_mse.history['loss'], 
        linestyle=line_styles[0], 
        color=colors[0], 
        marker=markers[2], 
        label="Train loss with Spatial Featrues")
plt.plot(hist_spat_mse.history['val_loss'], 
        linestyle=line_styles[1], 
        color=colors[0], 
        marker=markers[2], 
        label="Validation loss with Spatial Featrues")
plt.plot(control_train, 
        linestyle=line_styles[0], 
        color=colors[1], 
        marker=markers[1], 
        label="Train loss of control group")
plt.plot(control_val, 
        linestyle=line_styles[1], 
        color=colors[1], 
        marker=markers[1], 
        label="Validation loss of control group")
plt.ylabel('loss (MSE)')
plt.xlabel('epochs')
plt.yscale('log')
plt.legend(loc="right")

In [ ]:
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()

ax1.plot(hist_spat_mse.history['loss'], 
        linestyle=line_styles[0], 
        color=colors[0], 
        marker=markers[2], 
        label="Train loss with Spatial Featrues")
ax2.plot(hist_spat_mse.history['val_loss'], 
        linestyle=line_styles[1], 
        color=colors[0], 
        marker=markers[2], 
        label="Validation loss with Spatial Featrues")
ax1.plot(hist_batch_mse[2].history['loss'], 
        linestyle=line_styles[0], 
        color=colors[1], 
        marker=markers[1], 
        label="Train loss of control group")
ax2.plot(hist_batch_mse[2].history['val_loss'], 
        linestyle=line_styles[1], 
        color=colors[1], 
        marker=markers[1], 
        label="Validation loss of control group")

ax1.set_ylabel('train loss (MSE)')
ax2.set_ylabel('validation loss (MSE)')
ax1.set_xlabel('epochs')
ax1.set_yscale('log')
ax2.set_yscale('log')
ax1.set_ylim(5, 9)
ax2.set_ylim(4, 15)
ax1.set_xscale('log')
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='upper right')
plt.show()

In [ ]:
hist_spat_mse.history['loss'][-1] / hist_units_mse[4].history['loss'][-1]

In [ ]:
tf.keras.backend.clear_session()

#### Different Links

In [ ]:
ids = random.sample([i for i in range(0, NUM_LINKS)], 8)

In [ ]:
hist_spat_ids = []
test_loss_spat_ids = []

missing_test = []
missing_train = []

for id in ids:
    train1, test1 = createSeq(temp_data1[:, id, :], INPUT_LENGTH, PREDICT_LENGTH, TEST_LEN)
    train2, test2 = createSeq(temp_data2[:, id, :], INPUT_LENGTH, PREDICT_LENGTH, TEST_LEN)

    X_train = tf.concat([train1[0], train2[0]], 0)
    y_train = tf.concat([train1[1], train2[1]], 0)
    X_test = tf.concat([test1[0], test2[0]], 0)
    y_test = tf.concat([test1[1], test2[1]], 0)
    
    missing_test.append(tf.reduce_sum(tf.cast(tf.equal(y_test, 0), dtype=tf.int32)).numpy() / (y_test.shape[0] * y_test.shape[1]))
    missing_train.append(tf.reduce_sum(tf.cast(tf.equal(y_train, 0), dtype=tf.int32)).numpy() / (y_train.shape[0] * y_train.shape[1]))

    X_train, _, X_test= normalize(X_train, test=X_test)

    model = models.Sequential()
    model.add(layers.LSTM(units=200, return_sequences=False, input_shape=(INPUT_LENGTH, X_train.shape[-1])))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(units=PREDICT_LENGTH, activation='elu'))
    model.compile(optimizer='adam', loss=losses.MeanAbsolutePercentageError())
    
    hist_spat_ids.append(model.fit(X_train, y_train, epochs=1000, batch_size=120, callbacks=[earlystop, lr_scheduler], validation_split=0.2))
    test_loss_spat_ids.append(model.evaluate(X_test, y_test))

    tf.keras.backend.clear_session()

In [ ]:
tl = []
vl = []
for each in hist_spat_ids:
    tl.append(each.history['loss'][-1])
    vl.append(each.history['val_loss'][-1])

x = np.arange(len(ids))
width = 0.35

fig, ax = plt.subplots()
is_h = ax.bar(x, tl, width, label="train")
# not_h = ax.bar(x + width/2, vl, width, label="validation")

# ax.set_yscale('log')
ax.set_ylabel("Final Loss (MAPE)")
ax.set_xticks(x)
ax.set_xticklabels(ids)
ax.set_xlabel('Link ID')
# ax.legend()

In [ ]:
vl, tl, ids

## Globalised Design

In [6]:
temp_train = tf.concat([sortedData1[:-VALIDATION_LEN-TEST_LEN], sortedData2[:-VALIDATION_LEN-TEST_LEN]], 0)
temp_valid = tf.concat([sortedData1[-VALIDATION_LEN-TEST_LEN:-TEST_LEN], sortedData2[-VALIDATION_LEN-TEST_LEN:-TEST_LEN]], 0)
temp_test = tf.concat([sortedData1[-TEST_LEN:], sortedData2[-TEST_LEN:]], 0)

In [7]:
np.save('../datasets/global_temp_train.npy', temp_train)
np.save('../datasets/global_temp_val.npy', temp_valid)
np.save('../datasets/global_temp_test.npy', temp_test)

In [2]:
temp_train = np.load('../datasets/global_temp_train.npy')
temp_valid = np.load('../datasets/global_temp_val.npy')
temp_test = np.load('../datasets/global_temp_test.npy')

In [3]:
temp_train, temp_valid, temp_test = normalize(temp_train, temp_valid, temp_test)

In [4]:
earlystop = callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=10, verbose=1, restore_best_weights=True)

### STLSTM

In [5]:
spac_data = adj_matrix
stat_data = tf.reshape(links[:, 2], (links.shape[0], 1))

NameError: name 'adj_matrix' is not defined

In [ ]:
model = createSTLSTM(
    (INPUT_LENGTH, NUM_LINKS, FEATURE_COUNT), 
    (NUM_LINKS, NUM_LINKS), 
    (NUM_LINKS, SPAT_FEATURE_COUNT), 
    (PREDICT_LENGTH, NUM_LINKS)
)
model.summary()

In [ ]:
b_sizes = [120, 360, 720, 1440, 5040]

hist_stlstm_mse = []
time_taken_stlstm_mse = []
test_loss_stlstm_mse = []

hist_stlstm_mape = []
time_taken_stlstm_mape = []
test_loss_stlstm_mape = []

for b_size in b_sizes:
    train_generator = generateSTLSTMSeq(temp_train, INPUT_LENGTH, PREDICT_LENGTH, spac_data, stat_data, b_size)
    validation_generator = generateSTLSTMSeq(temp_valid, INPUT_LENGTH, PREDICT_LENGTH, spac_data, stat_data, b_size)
    test_generator = generateSTLSTMSeq(temp_test, INPUT_LENGTH, PREDICT_LENGTH, spac_data, stat_data, b_size)

    model = createSTLSTM(
        (INPUT_LENGTH, NUM_LINKS, FEATURE_COUNT), 
        (NUM_LINKS, NUM_LINKS), 
        (NUM_LINKS, SPAT_FEATURE_COUNT), 
        (PREDICT_LENGTH, NUM_LINKS)
    )
    
    start_time = time.time()
    hist_stlstm_mse.append(model.fit(
        train_generator, 
        steps_per_epoch=temp_train.shape[0] // b_size, 
        epochs=100, 
        callbacks=earlystop, 
        validation_data=next(validation_generator)))
    end_time = time.time()
    time_taken_stlstm_mse.append(end_time-start_time)

    test_loss_stlstm_mse.append(model.evaluate(test_generator, steps=temp_test.shape[0] // b_size))

    tf.keras.backend.clear_session()

    model = createSTLSTM(
        (INPUT_LENGTH, NUM_LINKS, FEATURE_COUNT), 
        (NUM_LINKS, NUM_LINKS), 
        (NUM_LINKS, SPAT_FEATURE_COUNT), 
        (PREDICT_LENGTH, NUM_LINKS), 
        losses.MeanAbsolutePercentageError()
    )
    
    start_time = time.time()
    hist_stlstm_mape.append(model.fit(
        train_generator, 
        steps_per_epoch=temp_train.shape[0] // b_size, 
        epochs=100, 
        callbacks=earlystop, 
        validation_data=next(validation_generator)))
    end_time = time.time()
    time_taken_stlstm_mape.append(end_time-start_time)
    
    test_loss_stlstm_mape.append(model.evaluate(test_generator, steps=temp_test.shape[0] // b_size))
    
    tf.keras.backend.clear_session()

In [ ]:
MSE_train = np.empty(len(hist_stlstm_mse))
MSE_valid = np.empty(len(hist_stlstm_mse))
for i, each in enumerate(hist_stlstm_mse):
    MSE_train[i] = each.history['loss'][-1]
    MSE_valid[i] = each.history['val_loss'][-1]

In [ ]:
MSE_train, MSE_valid, test_loss_stlstm_mse

In [ ]:
MAPE_train = np.empty(len(hist_stlstm_mape))
MAPE_valid = np.empty(len(hist_stlstm_mape))
for i, each in enumerate(hist_stlstm_mape):
    MAPE_train[i] = each.history['loss'][-1]
    MAPE_valid[i] = each.history['val_loss'][-1]

In [ ]:
MAPE_train, MAPE_valid, test_loss_stlstm_mape

### GCN-LSTM

In [13]:
spat_data = preprocessGraph(adj_matrix.numpy())
stat_data = links[:, 1:3]

In [15]:
np.save('../datasets/gcnlstm_spat.npy', spat_data)
np.save('../datasets/gcnlstm_stat.npy', stat_data)

In [6]:
spat_data = np.load('../datasets/gcnlstm_spat.npy')
stat_data = tf.convert_to_tensor(np.load('../datasets/gcnlstm_stat.npy'))

In [46]:
class GCN_LSTM(keras.Model):
    def __init__(
            self, 
            temporal_input_shape, 
            adj_norm, 
            static_input_shape, 
            output_shape, 
            ):
        super(GCN_LSTM, self).__init__()

        gcn_units = FEATURE_COUNT
        self.gcn = GCN(units=gcn_units, adj_norm=adj_norm)

        self.flatten_stat = layers.Flatten()
        self.repeat_stat = layers.RepeatVector(temporal_input_shape[0])
        self.reshape_stat = layers.Reshape(
            target_shape=(temporal_input_shape[0], static_input_shape[0], static_input_shape[1]), 
            name="Reshape_Static"
        )

        self.concatenate = layers.Concatenate(axis=-1, name="Concatenate")

        # self.conv2d = layers.Conv2D(
        #     filters=1, 
        #     kernel_size=(1, 1), 
        #     activation='relu',
        #     name="Conv2D"
        # )
        # self.reshape_all = layers.Reshape(
        #     target_shape=(temporal_input_shape[0], static_input_shape[0]),
        #     name="Reshape_All"
        # )

        # self.lstm = keras.Sequential([
        #     layers.GRU(units=200, dropout=0.2, return_sequences=True, activation='elu'),
        #     layers.GRU(units=200, dropout=0.2, activation='elu')
        #     ], name="LSTM_Layers"
        # )

        self.lstm = [
            keras.Sequential([
                layers.LSTM(units=50, dropout=0.2),
                layers.Dense(units=output_shape[0], activation='elu')
            ]) for _ in range(static_input_shape[0])]

        # self.dense = keras.Sequential([
        #     layers.Dense(units=1000, activation='elu'),
        #     layers.Dense(units=output_shape[0]*output_shape[1], activation='elu')
        #     ], name="Dense_Layers"
        # )

        # self.reshape_out = layers.Reshape(
        #     target_shape=(output_shape[0], output_shape[1]),
        #     name="Reshape_Output"
        # )
    
    def call(self, inputs, training=None):
        temp = inputs[0]    # [30, 132, 19]
        stat = inputs[1]    # [132, 1]
        
        # [132, 1] -> [30, 132, 1]
        stat = self.flatten_stat(stat)
        stat = self.repeat_stat(stat)
        stat = self.reshape_stat(stat)

        # [30, 132, 19] -> [30, 132, 19]
        # temp = self.gcn(temp)

        # [30, 132, 19] + [30, 132, 1]
        al = self.concatenate([temp, stat])

        # [30, 132, 20] -> [30, 132, 1] -> [30, 132]
        # al = self.conv2d(al)
        # al = self.reshape_all(al)

        # [30, 132] -> [100]
        # al = self.lstm(al)
        gru_outputs = []
        for i in range(132):
            link = al[:, :, i, :]
            gru_output = self.lstm[i](link)
            gru_outputs.append(gru_output)

        # al = tf.transpose(tf.convert_to_tensor(gru_outputs), perm=[1, 2, 0])
        al = tf.stack(gru_outputs, axis=2)
        # print(al.shape)

        # [100] -> [15*132] -> [15, 132]
        # al = self.dense(al)
        # al = self.reshape_out(al)

        return al

In [47]:
def createGCN_LSTM(temporal_shape, adj_norm, static_shape, output_shape, loss=losses.MeanSquaredError()):
    model = GCN_LSTM(
        temporal_input_shape=temporal_shape,
        adj_norm=adj_norm,
        static_input_shape=static_shape,
        output_shape=output_shape
    )

    model.build(input_shape=[(None, )+temporal_shape, (None, )+static_shape])
    model.compile(optimizer='adam', loss=loss)
    return model

In [48]:
model = createGCN_LSTM(
    (INPUT_LENGTH, NUM_LINKS, FEATURE_COUNT), 
    spat_data, 
    (NUM_LINKS, SPAT_FEATURE_COUNT), 
    (PREDICT_LENGTH, NUM_LINKS))
model.summary()

Model: "gcn_lstm_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gcn_2 (GCN)                 multiple                  0 (unused)
                                                                 
 flatten_2 (Flatten)         multiple                  0         
                                                                 
 repeat_vector_2 (RepeatVec  multiple                  0         
 tor)                                                            
                                                                 
 Reshape_Static (Reshape)    multiple                  0         
                                                                 
 Concatenate (Concatenate)   multiple                  0         
                                                                 
 sequential_264 (Sequential  (None, 15)                15165     
 )                                                      

In [49]:
BATCH_SIZE = 120

In [50]:
train_generator = generateGCN_LSTMSeq(temp_train, INPUT_LENGTH, PREDICT_LENGTH, stat_data, BATCH_SIZE)
validation_generator = generateGCN_LSTMSeq(temp_valid, INPUT_LENGTH, PREDICT_LENGTH, stat_data, BATCH_SIZE)
test_generator = generateGCN_LSTMSeq(temp_test, INPUT_LENGTH, PREDICT_LENGTH, stat_data, BATCH_SIZE)

In [51]:
earlystop = callbacks.EarlyStopping(monitor='loss', min_delta=0, patience=3, verbose=1, restore_best_weights=True)
lr_scheduler = callbacks.LearningRateScheduler(scheduler)
history = model.fit(
    train_generator, 
    steps_per_epoch=temp_train.shape[0] // BATCH_SIZE, 
    epochs=10, 
    callbacks=earlystop, 
    validation_data=next(validation_generator)
)

Epoch 1/10
924/924 [==============================] - 186s 108ms/step - loss: 0.0031 - val_loss: 0.0052
Epoch 2/10
924/924 [==============================] - 87s 95ms/step - loss: 1.4234e-04 - val_loss: 0.0028
Epoch 3/10
924/924 [==============================] - 87s 94ms/step - loss: 1.1672e-04 - val_loss: 0.0017
Epoch 4/10
924/924 [==============================] - 85s 92ms/step - loss: 1.0612e-04 - val_loss: 0.0012
Epoch 5/10
924/924 [==============================] - 86s 93ms/step - loss: 9.8800e-05 - val_loss: 8.1981e-04
Epoch 6/10
924/924 [==============================] - 85s 92ms/step - loss: 9.2564e-05 - val_loss: 6.1327e-04
Epoch 7/10
924/924 [==============================] - 85s 92ms/step - loss: 8.6959e-05 - val_loss: 4.8486e-04
Epoch 8/10
924/924 [==============================] - 85s 92ms/step - loss: 8.2767e-05 - val_loss: 4.1682e-04
Epoch 9/10
924/924 [==============================] - 85s 92ms/step - loss: 7.9659e-05 - val_loss: 3.7467e-04
Epoch 10/10
924/924 [=======

In [12]:
model.save_weights(model_path+'global_weights')

In [32]:
tf.keras.backend.clear_session()